In [ ]:
import os
import sys

from main import compress_and_compare, decompress_b64

from llama_cpp import Llama

In [ ]:
# choose a model

# throw metal warnings into the void
stderr_backup = sys.stderr
sys.stderr = open(os.devnull, 'w')

try: 
    llm = Llama.from_pretrained(
        repo_id="QuantFactory/SmolLM2-360M-GGUF",
        filename="*Q4_0.gguf",
        verbose=False,
        # logits_all=True,
        n_gpu_layers=-1,
        n_ctx=32768,
    )
finally:
    sys.stderr.close()
    sys.stderr = stderr_backup

In [ ]:
# compress text & compare against other algs
input_text = "The capital of France is Paris."

compress_and_compare(llm, input_text)

In [ ]:
# compress text & compare against other algs
input_text = "The capital of France is Paris. The capital of Germany is Berlin. The capital of Italy is Rome. The capital of Spain is Madrid. The capital of Portugal is Lisbon. The capital of Belgium is Brussels. The capital of Netherlands is Amsterdam. The capital of Luxembourg is Luxembourg. The capital of Monaco is Monaco. The capital of Andorra is Andorra la Vella. The capital of San Marino is San Marino. The capital of Vatican City is Vatican City. The capital of Liechtenstein is Vaduz."

compress_and_compare(llm, input_text)

In [ ]:
# compress text & compare against other algs
input_text = """全会充分肯定党的二十届三中全会以来中央政治局的工作。一致认为，中央政治局认真落实党的二十大和二十届历次全会精神"""
compress_and_compare(llm, input_text)

In [ ]:
# compress text & compare against other algs
input_text = """"In information theory, data compression, source coding,[1] or bit-rate reduction is the process of encoding information using fewer bits than the original representation.[2] Any particular compression is either lossy or lossless. Lossless compression reduces bits by identifying and eliminating statistical redundancy. No information is lost in lossless compression. Lossy compression reduces bits by removing unnecessary or less important information.[3] Typically, a device that performs data compression is referred to as an encoder, and one that performs the reversal of the process (decompression) as a decoder.
        The process of reducing the size of a data file is often referred to as data compression. In the context of data transmission, it is called source coding: encoding is done at the source of the data before it is stored or transmitted.[4] Source coding should not be confused with channel coding, for error detection and correction or line coding, the means for mapping data onto a signal.
        Data compression algorithms present a space–time complexity trade-off between the bytes needed to store or transmit information, and the computational resources needed to perform the encoding and decoding. The design of data compression schemes involves balancing the degree of compression, the amount of distortion introduced (when using lossy data compression), and the computational resources or time required to compress and decompress the data.[5] """

compress_and_compare(llm, input_text)

In [ ]:
decompress_b64(llm, "AAAAb/32Ms67JZFXnlOdrCN9ZUU4uS3AFjIbJA==")

In [ ]:
decompress_b64(llm, "AAAAb/32Ms67JZFXnlOdsCN9ZUU4uS3AFjIbJA==")